In [1]:
import random
import cv2
from pathlib import Path
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
from torchvision import models
from torchinfo import summary
import numpy as np
import torchmetrics
from src.data.dataset import ProcessedWoundDataset
from src.data.make_dataset import make_dataset1, make_dataset2
from src.utils import empty_directory
from src.models.mobilnetv2 import MobileNetV2withDecoder
from config.defaults import get_cfg_defaults

In [2]:
base_model = models.mobilenet_v2(weights='MobileNet_V2_Weights.IMAGENET1K_V1')

no_classifier_model = torch.nn.Sequential(*(list(base_model.children())[:-1]))

stripped_model = torch.nn.Sequential(*(list(no_classifier_model[0].children())[:-1]))

model = MobileNetV2withDecoder(stripped_model, classes=1)

In [3]:
datasets = {}
datasets['train'] = ProcessedWoundDataset('./data/processed/train/')
datasets['val'] = ProcessedWoundDataset('./data/processed/val/')
datasets['test'] = ProcessedWoundDataset('./data/processed/test/')

dataloaders = {x: DataLoader(dataset=datasets[x], batch_size=16, shuffle=True,
                                num_workers=0) for x in ['train','val', 'test']}


In [4]:
for image, mask in dataloaders['train']:
    print(image.dtype, mask.dtype)
    break

torch.float32 torch.float32


In [5]:
pred = model(image)
pred.shape

/Users/yk30ulej/anaconda3/envs/swd/lib/python3.8/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


torch.Size([16, 1, 224, 224])

In [6]:
loss = nn.BCELoss()
loss(pred, mask)

tensor(0.8058, grad_fn=<BinaryCrossEntropyBackward0>)